In [20]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os, sys
from keras.models import Model
import csv
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def load_image(root_path, height, width, classes):
    data, labels = [], []
    dirs = os.listdir(root_path)
    if '.DS_Store' in dirs:
        dirs.remove('.DS_Store')
    for d in dirs:
        path = os.path.join(root_path, d)
        img_count = os.listdir(path)
        for img in img_count:
            try: 
                img_path = os.path.join(path, img)
                image = cv2.imread(img_path)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((width, height))
                data.append(np.array(size_image))
                labels.append(d)
            except AttributeError:
                print(" ")
    
    Cells = np.array(data)
    labels = np.array(labels)
    
    s = np.arange(Cells.shape[0])
    np.random.seed(classes)
    np.random.shuffle(s)
    Cells = Cells[s]
    labels = labels[s]
    
    X = Cells.astype('float32')/255
    print(X.shape)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    y = to_categorical(y, classes)
    return X, y


def data_processing(data_path, height, width, classes):
    train_path = os.path.join(data_path, 'NonAugmentedTrain')
    val_path = os.path.join(data_path, 'ValData')
    X_train, y_train = load_image(train_path, height, width, classes)
    X_val, y_val = load_image(val_path, height, width, classes)
    return X_train, y_train, X_val, y_val




In [21]:
from TP_Net_2 import get_flops, piecewise5, custom_network
import keras
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

if __name__ == '__main__':
    height = 400
    width = 300
    channels = 3
    classes = 4
    ratio_train = 0.8
    ratio_val = 0.2
    save_path = './training_info.csv'
    pre_trained_model_path = ''
    model_save_path = './test2_model.h5'
    data_path = './covid19-detection-xray-dataset'
    
    if ratio_train + ratio_val > 1:
        print('Train/eval splitting failed')
        exit(0)
    X_train, y_train, X_val, y_val = data_processing(data_path,
                                                     height, 
                                                     width,  
                                                     classes)
        
    get_custom_objects().update({'piecewise5': Activation(piecewise5)})
    input_shape = X_train.shape[1:]

    sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
    model = custom_network(height, width, classes, pre_trained_model_path)
    
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    
    epochs = 5
    hist1 = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
    model.save(model_save_path)  # should end with .h5 or .hdf5
    
    history = hist1.history
    
    score = model.evaluate(X_val, y_val, verbose=0)
    score2 = model.evaluate(X_train, y_train, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print('Training loss:', score2[0])
    print('Training accuracy:', score2[1])



(2002, 400, 300, 3)
(988, 400, 300, 3)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 400, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 400, 300, 16) 64          input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 400, 300, 32) 4640        conv2d_23[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_14 (MaxPooling2D) (None, 200, 150, 32) 0           conv2d_24[0][0]                  
_____________________________________________________